In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE151181"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE151181"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE151181.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE151181.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE151181.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['histological variant: Classical', 'histological variant: Follicular', 'histological variant: NA', 'histological variant: non-neoplastic thyroid'], 1: ['tissue type: Primary tumor', 'tissue type: synchronous lymph node metastasis', 'tissue type: lymph node metastasis post RAI', 'tissue type: lymph node metastasis_2 post RAI', 'tissue type: lymph node metastasis_1 post RAI', 'tissue type: non-neoplastic thyroid'], 2: ['collection before/after rai: Before', 'collection before/after rai: After'], 3: ['patient id: pt_1', 'patient id: pt_2', 'patient id: pt_3', 'patient id: pt_5', 'patient id: pt_7', 'patient id: pt_8', 'patient id: pt_11', 'patient id: pt_12', 'patient id: pt_13', 'patient id: p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the title and summary, this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait data, looking at the characteristics:
# We can use "patient rai responce" (row 4) which indicates whether the patient is Avid or Refractory for radioiodine
trait_row = 4

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert thyroid cancer trait (radioiodine response) to binary format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Avid = 0 (responsive to radioiodine treatment)
    # Refractory = 1 (non-responsive to radioiodine treatment)
    if value.lower() == 'avid':
        return 0
    elif value.lower() == 'refractory':
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous format"""
    # Not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary format"""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save the cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM4567964': [0.0], 'GSM4567965': [0.0], 'GSM4567966': [0.0], 'GSM4567967': [0.0], 'GSM4567968': [1.0], 'GSM4567969': [1.0], 'GSM4567970': [1.0], 'GSM4567971': [1.0], 'GSM4567972': [1.0], 'GSM4567973': [1.0], 'GSM4567974': [1.0], 'GSM4567975': [1.0], 'GSM4567976': [1.0], 'GSM4567977': [1.0], 'GSM4567978': [1.0], 'GSM4567979': [1.0], 'GSM4567980': [1.0], 'GSM4567981': [1.0], 'GSM4567982': [1.0], 'GSM4567983': [1.0], 'GSM4567984': [1.0], 'GSM4567985': [1.0], 'GSM4567986': [1.0], 'GSM4567987': [1.0], 'GSM4567988': [1.0], 'GSM4567989': [1.0], 'GSM4567990': [1.0], 'GSM4567991': [1.0], 'GSM4567992': [1.0], 'GSM4567993': [1.0], 'GSM4567994': [1.0], 'GSM4567995': [1.0], 'GSM4567996': [1.0], 'GSM4567997': [1.0], 'GSM4567998': [1.0], 'GSM4567999': [1.0], 'GSM4568000': [0.0], 'GSM4568001': [0.0], 'GSM4568002': [1.0], 'GSM4568003': [1.0], 'GSM4568004': [1.0], 'GSM4568005': [1.0], 'GSM4568006': [1.0], 'GSM4568007': [1.0], 'GSM4568008': [1.0], 'GSM4568009': [1.0], 'GSM456801

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE151181/GSE151181_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE151181/GSE151181-GPL21575_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (62976, 47)
First 20 gene/probe identifiers:
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']


### Step 4: Gene Identifier Review

In [5]:
# Reviewing gene identifiers in the dataset
# These don't appear to be standard human gene symbols
# They look like probe IDs from a microarray platform (GPL23159) that need mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check for columns that might contain gene information
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Correctly identify the columns for probe ID and gene symbols
gene_id_col = 'ID'  # This is the probe identifier column
gene_symbol_col = 'GENE_SYMBOL'  # Corrected to match the actual column name in the data

# Verify columns exist before mapping
if gene_id_col in gene_annotation.columns and gene_symbol_col in gene_annotation.columns:
    # Create the mapping using the library function
    mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    missing_cols = []
    if gene_id_col not in gene_annotation.columns:
        missing_cols.append(gene_id_col)
    if gene_symbol_col not in gene_annotation.columns:
        missing_cols.append(gene_symbol_col)
    print(f"\nError: The following columns are missing from the annotation data: {missing_cols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'SPOT_ID', 'CONTROL_TYPE', 'miRNA_ID', 'GENE_SYMBOL', 'GENE_NAME', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': ['328', '326', '324', '322', '320'], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}

Complete sample of a few rows:
  ID  COL  ROW              SPOT_ID CONTROL_TYPE miRNA_ID GENE_SYMBOL GENE_NAME ACCESSION_STRING CHROMOSOMAL_LOCATION
0  1  192  328  miRNABrightCorner30          pos      NaN         NaN       NaN              NaN                  NaN
1  2  192  326                Blank          pos      NaN         NaN       NaN 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns store probe IDs and gene symbols in the annotation data
# Print a sample of the gene annotation data to understand the structure
gene_annotation = get_gene_annotation(soft_file)

# Get a sample of gene expression data IDs
gene_expr_data = get_genetic_data(matrix_file)
print("Gene expression data ID format:", gene_expr_data.index[:5].tolist())

# Sample annotation data to understand the structure
print("Sample of gene annotation data:")
print(gene_annotation[['ID', 'miRNA_ID', 'GENE_SYMBOL']].head(10))

# Check how many non-null values we have for miRNA_ID vs GENE_SYMBOL
print("\nNon-null count for potential ID columns:")
print(f"miRNA_ID: {gene_annotation['miRNA_ID'].notna().sum()}")
print(f"GENE_SYMBOL: {gene_annotation['GENE_SYMBOL'].notna().sum()}")

# Based on the observations from Step 5, this appears to be miRNA data, not gene expression
# Let's check for matches between expression data IDs and annotation IDs
expr_ids = set(gene_expr_data.index.astype(str))
ann_ids = set(gene_annotation['ID'].astype(str))
overlap = expr_ids.intersection(ann_ids)
print(f"\nDirect matches between expression IDs and annotation IDs: {len(overlap)}")

# Create a mapping dataframe that considers both miRNA_ID and GENE_SYMBOL
mapping_data = gene_annotation[['ID', 'miRNA_ID']].copy()
mapping_data.rename(columns={'miRNA_ID': 'Gene'}, inplace=True)
mapping_data = mapping_data.dropna(subset=['Gene'])

# Print a sample of the mapping data
print("\nSample of mapping data (before conversion):")
print(mapping_data.head(10))

# Check how many expression IDs can be mapped
mappable_ids = set(mapping_data['ID'].astype(str))
mappable_overlap = expr_ids.intersection(mappable_ids)
print(f"\nOverlap between expression IDs and mappable IDs: {len(mappable_overlap)}")

# Convert IDs to strings for proper matching
mapping_data['ID'] = mapping_data['ID'].astype(str)

# Apply the gene mapping function
try:
    gene_data = apply_gene_mapping(gene_expr_data, mapping_data)
    print(f"\nShape of the gene expression data after mapping: {gene_data.shape}")
    print("First few gene symbols after mapping:")
    print(gene_data.index[:10].tolist() if not gene_data.empty else "No genes mapped")
except Exception as e:
    print(f"\nError during gene mapping: {e}")
    # Since this appears to be miRNA data rather than gene expression data,
    # we should update is_gene_available
    is_gene_available = False
    gene_data = pd.DataFrame()
    print("This dataset contains miRNA data rather than gene expression data.")
    print("Setting is_gene_available to False.")

# Even if the mapping fails, let's reassess whether this is actually gene expression data
if gene_data.empty or len(gene_data.index) < 100:  # Arbitrary threshold for minimum mapped genes
    is_gene_available = False
    print("\nThis dataset does not contain sufficient gene expression data for our analysis.")
    print("Setting is_gene_available to False.")
    
    # Update this information in the JSON file
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=(trait_row is not None)
    )

Gene expression data ID format: ['1', '2', '3', '4', '5']
Sample of gene annotation data:
   ID miRNA_ID GENE_SYMBOL
0   1      NaN         NaN
1   2      NaN         NaN
2   3      NaN         NaN
3   4      NaN         NaN
4   5      NaN         NaN
5   6      NaN         NaN
6   7      NaN         NaN
7   8      NaN         NaN
8   9      NaN         NaN
9  10      NaN         NaN

Non-null count for potential ID columns:
miRNA_ID: 75332
GENE_SYMBOL: 75332



Direct matches between expression IDs and annotation IDs: 62976

Sample of mapping data (before conversion):
    ID             Gene
10  11     hsa-miR-3132
12  13  hsa-miR-449c-5p
13  14     hsa-miR-4481
14  15  hsa-miR-6788-5p
15  16  hsa-miR-514a-3p
16  17     hsa-miR-3907
17  18  hsa-miR-516b-5p
20  21  hsa-miR-7153-3p
22  23  hsa-miR-7843-3p
23  24  hsa-miR-6845-3p

Overlap between expression IDs and mappable IDs: 50980



Shape of the gene expression data after mapping: (0, 47)
First few gene symbols after mapping:
No genes mapped

This dataset does not contain sufficient gene expression data for our analysis.
Setting is_gene_available to False.
